# Introduction to Model Predictive Control

In [8]:
rm(list = ls())

In [9]:
library(MatrixTSA)
library(tibble)
library(tidyverse)
options(tibble.width = Inf)

Warning message:
“package ‘tibble’ was built under R version 3.5.2”── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tidyr   0.8.3     ✔ dplyr   0.8.3
✔ readr   1.3.0     ✔ stringr 1.4.0
✔ ggplot2 3.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘ggplot2’ was built under R version 3.5.2”Warning message:
“package ‘tidyr’ was built under R version 3.5.2”Warning message:
“package ‘purrr’ was built under R version 3.5.2”Warning message:
“package ‘dplyr’ was built under R version 3.5.2”Warning message:
“package ‘stringr’ was built under R version 3.5.2”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_factor() masks scales::col_factor()
✖ purrr::discard()    masks scales::discard()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()


In [11]:
setwd("~/GitHub/MatrixTSA")
source("./examples/MPC/src/asHours.R")
source("./examples/MPC/src/asP.R")
source("./examples/MPC/src/prbs.R")
source("./examples/MPC/src/sim_building.R")

In [20]:
ti <- read_csv(
  "./data/data_mpc.csv",
  skip = 1,
  col_names = c("timedate", "Y1", "Y2", "Ta", "Gv", "Ph1", "Ph2")
)
ti <- ti %>%
  mutate(timedate = asP(.$timedate)) %>%
  mutate(t = asHours(.$timedate - .$timedate[1]))

Parsed with column specification:
cols(
  timedate = col_datetime(format = ""),
  Y1 = col_double(),
  Y2 = col_double(),
  Ta = col_double(),
  Gv = col_double(),
  Ph1 = col_double(),
  Ph2 = col_double()
)


The grammar rules of a programming language are rigid. In Python, the `*` symbol cannot appear twice in a row. The computer will not try to interpret an expression that differs from its prescribed expression structures. Instead, it will show a `SyntaxError` error. The *Syntax* of a language is its set of grammar rules, and a `SyntaxError` indicates that an expression structure doesn't match any of the rules of the language.

In [21]:
ti_est <- tibble(
  name = c(
    "T1a0", "T1m0", "C1a", "C1m", "R1a",
    "R1m", "A1w", "p1", "p1a", "p1m", "e11"
  ),
  init = c(25, 25, 6, 12, 10, 1, 1, 0.5, 1, 1, -1),
  lb = c(0, 0, 1E-5, 1, 1, 1E-10, 1E-10, 0, -30, -30, -50),
  up = c(35, 35, 20, 50, 80, 10, 10, 1, 10, 10, 10)
)
li_mod <- list()
li_mod[[1]] <- set_mod_ctsm(
  c_expr_sys = c(
    d(T1a) ~ (1 / (C1a * R1m) * (T1m - T1a) + 1 / (C1a * R1a) * (Ta - T1a) + 1 / C1a * Ph1 + p1 * A1w / C1a * Gv) * d(t) + exp(p1a) / C1a * d(w1a),
    d(T1m) ~ (1 / (C1m * R1m) * (T1a - T1m) + (1 - p1) * A1w / C1m * Gv) * d(t) + exp(p1m) / C1m * d(w1m)
  ),
  expr_obs = Y1 ~ T1a,
  expr_var = Y1 ~ exp(e11),
  c_input = c("Ta", "Ph1", "Gv"),
  ti_est = ti_est
)

Warning message in eval(expr, envir, enclos):
“internal error -3 in R_decompress1”

ERROR: Error in eval(expr, envir, enclos): lazy-load database '/Library/Frameworks/R.framework/Versions/3.5/Resources/library/MatrixTSA/R/MatrixTSA.rdb' is corrupt


Small changes to an expression can change its meaning entirely. Below, the space between the `*`'s has been removed. Because `**` appears between two numerical expressions, the expression is a well-formed *exponentiation* expression (the first number raised to the power of the second: 3 times 3 times 3 times 3). The symbols `*` and `**` are called *operators*, and the values they combine are called *operands*.

In [5]:
3 ** 4

81

**Common Operators.** Data science often involves combining numerical values, and the set of operators in a programming language are designed to so that expressions can be used to express any sort of arithmetic. In Python, the following operators are essential.

| Expression Type | Operator | Example    | Value     |
|-----------------|----------|------------|-----------|
| Addition        | `+`      | `2 + 3`    | `5`       |
| Subtraction     | `-`      | `2 - 3`    | `-1`      |
| Multiplication  | `*`      | `2 * 3`    | `6`       |
| Division        | `/`      | `7 / 3`    | `2.66667` |
| Remainder       | `%`      | `7 % 3`    | `1`       |
| Exponentiation  | `**`     | `2 ** 0.5` | `1.41421` |

Python expressions obey the same familiar rules of *precedence* as in algebra: multiplication and division occur before addition and subtraction. Parentheses can be used to group together smaller expressions within a larger expression.

In [4]:
1 + 2 * 3 * 4 * 5 / 6 ** 3 + 7 + 8 - 9 + 10

17.555555555555557

In [3]:
1 + 2 * (3 * 4 * 5 / 6) ** 3 + 7 + 8 - 9 + 10

2017.0

This chapter introduces many types of expressions. Learning to program involves trying out everything you learn in combination, investigating the behavior of the computer. What happens if you divide by zero? What happens if you divide twice in a row? You don't always need to ask an expert (or the Internet); many of these details can be discovered by trying them out yourself. 